In [1]:
import pathlib
from datetime import datetime
from typing import List, Tuple, Union, Dict

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Column

from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window


In [2]:
# master configuration to use only 4 CPU cores
spark = SparkSession.builder.master("local[4]").getOrCreate()

# basic configuration to use only a reasonable number of partitions
spark.conf.set("spark.sql.shuffle.partition", 4)

# configuration to work in UTC
spark.conf.set("spark.sql.session.timeZone", "UTC")


24/01/11 17:38:12 WARN Utils: Your hostname, Emilianos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.130 instead (on interface en0)
24/01/11 17:38:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/01/11 17:38:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
path = "/Users/emilianofrigo/Downloads/amazon_financial_events_financial_shipment_events_part-00008-e7ec740a-599b-4042-8f3c-19dbc78360f5-c000.snappy.parquet"

df = spark.read.parquet(path)


In [5]:
df.printSchema()

root
 |-- region_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- marketplace_id: string (nullable = true)
 |-- posted_dt: date (nullable = true)
 |-- report_id: string (nullable = true)
 |-- raw_data: struct (nullable = true)
 |    |-- AmazonOrderId: string (nullable = true)
 |    |-- MarketplaceName: string (nullable = true)
 |    |-- PostedDate: string (nullable = true)
 |    |-- SellerOrderId: string (nullable = true)
 |    |-- ShipmentFeeList: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- FeeAmount: struct (nullable = true)
 |    |    |    |    |-- CurrencyAmount: double (nullable = true)
 |    |    |    |    |-- CurrencyCode: string (nullable = true)
 |    |    |    |-- FeeType: string (nullable = true)
 |    |-- ShipmentItemList: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- CostOfPointsGranted: struct (nullable = true)
 |    |    |    |    |-- Currenc

24/01/11 18:39:36 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 907152 ms exceeds timeout 120000 ms
24/01/11 18:39:36 WARN SparkContext: Killing executors is not supported by current scheduler.


In [18]:
df.show(1, vertical=True, truncate=False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [40]:
df.count()

4120

In [41]:
# Repartition the DataFrame to a single partition (1)
df = df.repartition(1)

# Write the DataFrame to a JSON file with a single output file
df.write.mode("overwrite").json("output.json")

In [38]:
df_selected = (
    df
    .filter("raw_data.product_id = 'AMP'")
    .limit(3)
)


In [39]:
df_selected.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+
|product_id|           system_id|          updated_at| report_ingested_ts|raw_data_json_string|            raw_data|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+
|       AMP|s-141b0b9f-9db9-4...|2023-11-02 12:29:...|2023-11-06 01:02:42|{"brand":"Amzpet"...|{null, null, null...|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+



In [36]:
df_selected.count()

1

In [34]:
# Repartition the DataFrame to a single partition (1)
df_selected = df_selected.repartition(1)

# Write the DataFrame to a JSON file with a single output file
df_selected.write.mode("overwrite").json("output.json")